In [34]:
%cd ~/Documents/cvd-predictor/
# from sklearnex import patch_sklearn
# patch_sklearn()
import polars as pl
from CVD.utils import get_metrics, encode_data
from CVD.visuals import *
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from plotly.graph_objs._figure import Figure
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import pickle

C:\Users\Lorenzo\Documents\cvd-predictor


In [35]:
csv_path = "data/intermediate/heart_cdc_2023.csv"
parquet_path = "data/intermediate/heart_cdc_2023.parquet"
pl.read_csv(csv_path).write_parquet(parquet_path)

Non-essential or redundant features are removed under the hood using select_features(). Selected categorical columns into binary or numerical formats  are encoded using predefined mappings. 

Additionally, BMI is recomputed from height and weight. After encoding, rows with missing values are dropped.

In [36]:
df: pl.DataFrame = pl.read_parquet(parquet_path)
df: pl.DataFrame = encode_data(df).drop_nulls()
df.write_parquet("data/intermediate/heart_cdc_2023_cleaned.parquet")
df

Sex,GeneralHealth,PhysicalActivities,HadAsthma,HadCOPD,HadKidneyDisease,HadArthritis,HadDiabetes,SmokerStatus,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HaveHighCholesterol,Sensory Impairments,Vaccinated,Mobility,CVD
i64,i64,i8,i8,i8,i8,i64,i64,i8,i64,f64,f64,f64,i8,i64,i64,i64,i64,i64
0,2,1,1,0,0,2,1,0,13,1.6,78.02,30.476562,0,0,0,0,1,0
0,2,0,1,0,0,1,3,0,13,1.45,59.87,28.475624,0,1,1,1,0,0
0,4,0,0,0,0,1,3,0,13,1.63,58.97,22.195039,0,1,1,1,1,0
0,2,0,1,0,0,1,3,0,12,1.68,77.11,27.32072,0,0,0,1,1,0
0,4,0,1,0,0,1,1,0,12,1.73,77.11,25.764309,1,0,0,1,1,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,2,1,0,0,0,2,3,0,6,1.75,66.22,21.622857,0,0,0,1,0,0
1,3,0,1,0,0,2,3,0,10,1.93,108.86,29.224946,1,1,0,0,0,0
0,2,1,1,0,0,2,3,0,3,1.65,68.04,24.991736,0,0,0,0,0,0


In [37]:
fig: Figure = correlation_matrix(df, ["Sex"], "no")
fig.show()

Although our best-performing models LightGBM and XGBoost are tree-based and do not require feature scaling, we apply standardization to the full dataset. This is done to ensure better runtime inference.

In [ ]:
X: pl.DataFrame = df.drop(["CVD"])
y: pl.Series = df["CVD"]

scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X.to_numpy())``

# pickle.dump(scaler, open("data/training/Scaler.pkl", "wb"))

Experimenting with different standard train/test splits 80/20 gives the best performance without overfitting, etc

In [39]:
X_train, X_test, y_train, y_test = train_test_split(
	X, y, test_size=0.2, random_state=42, stratify=y
)

In [40]:
count_plot(y_train)

The dataset is imbalanced, and due to Trump's recent orders to stop providing medical data, it can't be collated with other sets.  
Source: https://publichealth.jhu.edu/2025/the-consequences-of-the-us-withdrawal-from-the-who

In [41]:
models: dict = {
    "Logistic Regression": LogisticRegression(max_iter=10_000, random_state=42),
    "Naive Bayes": GaussianNB(),
    "SVM": SVC(probability=True),  # SVM with probability take my word this performs bad and runs long
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    "LightGBM": LGBMClassifier(n_estimators=1_000, random_state=42, n_jobs=-1),
    "XGBoost": XGBClassifier(
        n_estimators=1_000,
        eval_metric="map",
        random_state=42,
        tree_method="hist",
        device="cuda",
        verbosity=2,
    ),
    "MLP": MLPClassifier(
        hidden_layer_sizes=(64, 32),
        activation="relu",
        solver="adam",
        alpha=1e-4,
        batch_size="auto",
        learning_rate="adaptive",
        max_iter=200,
        random_state=42,
    ),
}

In [ ]:
results: list[dict] = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_prob: np.ndarray = model.predict_proba(X_test)[:, 1]
    y_pred_binary: np.ndarray = (y_pred_prob >= 0.5).astype(int)
    print(classification_report(y_test, y_pred_binary))
    results.append(get_metrics(y_test, y_pred_binary, name))
    # pickle.dump(model, open(f"data/training/{name}.pkl", "wb"))
    # plot_feature_importances(model, X, 15, name)

              precision    recall  f1-score   support

           0       0.91      0.96      0.94     56356
           1       0.57      0.35      0.43      8202

    accuracy                           0.88     64558
   macro avg       0.74      0.66      0.69     64558
weighted avg       0.87      0.88      0.87     64558

              precision    recall  f1-score   support

           0       0.94      0.86      0.90     56356
           1       0.39      0.63      0.48      8202

    accuracy                           0.83     64558
   macro avg       0.67      0.74      0.69     64558
weighted avg       0.87      0.83      0.84     64558

              precision    recall  f1-score   support

           0       0.92      0.97      0.94     56356
           1       0.68      0.39      0.50      8202

    accuracy                           0.90     64558
   macro avg       0.80      0.68      0.72     64558
weighted avg       0.89      0.90      0.89     64558

              preci

In [43]:
results_df = pl.DataFrame(results)
results_df

Model,Accuracy,Precision,Recall,Specificity,F1 Score,ROC AUC
str,f64,f64,f64,f64,f64,f64
"""Logistic Regression""",0.884306,0.573081,0.350402,0.962009,0.434894,0.656206
"""Naive Bayes""",0.829006,0.392105,0.628505,0.858187,0.482927,0.743346
"""SVM""",0.899393,0.681634,0.390515,0.973454,0.496551,0.681984
"""Random Forest""",0.908516,0.671241,0.548647,0.960891,0.603784,0.754769
"""LightGBM""",0.959261,0.901788,0.762375,0.987916,0.826242,0.875146
"""XGBoost""",0.953313,0.855635,0.760912,0.981315,0.805498,0.871114
"""MLP""",0.903668,0.642436,0.545233,0.955834,0.589857,0.750534


**Summary of Results**  
LightGBM and XGBoost achieved the highest performance (≈95% accuracy, F1 ≈ 0.82 – 0.83, ROC AUC ≈ 0.88). Random Forest and the MLP followed with moderate accuracy (≈90%) and F1 scores around 0.60. Logistic Regression and Naive Bayes exhibited imbalanced trade-offs (high specificity or recall) and the lowest F1 values.

**Comparison with Other Approaches**  
Neural networks and MLPs classifiers were also evaluated but did not match the performance of gradient-boosting models. Boosted trees better accommodate imbalanced, sparse lifestyle data, capture nonlinear interactions without extensive feature engineering, and produce well-calibrated probability estimates with minimal hyperparameter tuning—advantages that translate into superior F1 scores as observed.